In [1]:
%matplotlib inline

import numpy as np
import matplotlib as mpl
from matplotlib import pyplot as plt
import seaborn as sns
sns.set()

import pandas as pd
pd.options.display.float_format = '{:.2f}'.format

import json
import glob
import os
import re

In [2]:
paths = glob.glob("../exp/bidir_features/*.json")
data = [json.load(open(path)) for path in paths]

In [3]:
queries = pd.DataFrame.from_records([{
    **algo,
    'graph': { 'europe': 'DIMACs Europe', 'osm_ger': 'OSM Germany', 'osm_europe': 'OSM Europe' }[[x for x in run['args'][1].split('/') if x != ''][-1]],
    'experiment': exp['experiment'],
    'factor': exp['factor'],
    'probability': exp.get('probability', 1),
    'speed': exp.get('speed', 0),
    'potential': run['potential'],
    'num_queue_pops': algo['num_queue_pops'] + algo.get('core_search', {}).get('num_queue_pops', 0),
    'num_queue_pushs': algo['num_queue_pushs'] + algo.get('core_search', {}).get('num_queue_pushs', 0),
    'num_relaxed_arcs': algo['num_relaxed_arcs'] + algo.get('core_search', {}).get('num_relaxed_arcs', 0),
    }
    for run in data for exp in run['experiments'] for algo in exp['algo_runs']])

queries['choose_direction_strategy'] = queries['choose_direction_strategy'].fillna('unidir')
queries['bidir_pot'] = queries['bidir_pot'].fillna('unidir')
queries['improved_pruning'] = queries['improved_pruning'].fillna('unidir')
queries['num_queue_pushs_k'] = queries['num_queue_pushs'] / 1000

In [4]:
pd.set_option('display.max_rows', 500)

In [5]:
queries = queries.loc[lambda x: x['graph'] == 'OSM Germany']

In [6]:
queries.groupby(['potential', 'experiment', 'factor', 'probability', 'algo', 'bidir_pot', 'choose_direction_strategy', 'improved_pruning'])[['running_time_ms', 'num_queue_pushs_k']].mean().unstack(0)

running_time_ms  \
potential                                                                                                                                                  ALT   
experiment                   factor probability algo                                      bidir_pot choose_direction_strategy improved_pruning                   
probabilistic_scale_by_speed 1.50   0.00        Bidrectional Dijkstra Query               Average   alternating               False                     959.54   
                                                                                                                              True                      968.97   
                                                                                                    min_key                   False                     985.47   
                                                                                                                              True                      991.69   
                                                                                          Symmetric alternating               False                    2775.45   
                                                                                                                              True                      906.57   
                                                                                                    min_key                   False                    2751.36   
                                                                                                                              True                      943.24   
                                                Dijkstra Query                            unidir    unidir                    unidir                   1088.41   
                                                Virtual Topocore Bidirectional Core Query Average   alternating               False                     270.89   
                                                                                                                              True                      270.74   
                                                                                                    min_key                   False                     276.73   
                                                                                                                              True                      278.15   
                                                                                          Symmetric alternating               False                     922.73   
                                                                                                                              True                      274.22   
                                                                                                    min_key                   False                     912.14   
                                                                                                                              True                      287.33   
                                                Virtual Topocore Component Query          unidir    unidir                    unidir                    327.19   
                                    1.00        Bidrectional Dijkstra Query               Average   alternating               False                      73.77   
                                                                                                                              True                       74.59   
                                                                                                    min_key                   False                     117.29   
                                                                                                                              True                      118.55   
                                                                                          Sym

### Effectiveness new pruning

In [56]:
pruning = queries.loc[lambda x: x['choose_direction_strategy'] != 'min_key'] \
    .loc[lambda x: x['bidir_pot'] != 'unidir'] \
    .groupby(['potential', 'experiment', 'factor', 'probability', 'algo', 'bidir_pot', 'improved_pruning'])[['running_time_ms', 'num_queue_pushs_k']].mean() \
    .unstack(0) \
    .loc[('weight_scale', 1.05, 1.00)] \
    .reindex(columns=['Zero', 'ALT', 'CH', 'CCH', 'Oracle'], level=1)

pruning[('num_queue_pushs_k', '(C)CH/Oracle')] = pruning[('num_queue_pushs_k', 'CH')]
pruning = pruning.drop([('num_queue_pushs_k', 'CH'), ('num_queue_pushs_k', 'CCH'), ('num_queue_pushs_k', 'Oracle')], axis = 'columns')

pruning = pruning.rename(columns={
        'running_time_ms': 'Running time [ms]',
        'num_queue_pushs_k': 'Queue pushs [$\\cdot 10^3$]',
    }) \
    .reset_index()

pruning

algo  bidir_pot  \
potential                                                         
0                        Bidrectional Dijkstra Query    Average   
1                        Bidrectional Dijkstra Query    Average   
2                        Bidrectional Dijkstra Query  Symmetric   
3                        Bidrectional Dijkstra Query  Symmetric   
4          Virtual Topocore Bidirectional Core Query    Average   
5          Virtual Topocore Bidirectional Core Query    Average   
6          Virtual Topocore Bidirectional Core Query  Symmetric   
7          Virtual Topocore Bidirectional Core Query  Symmetric   

          improved_pruning Running time [ms]                              \
potential                               Zero    ALT     CH    CCH Oracle   
0                    False           1441.41 126.46  62.61  53.91  37.29   
1                     True           1451.96 128.20  62.48  54.28  38.89   
2                    False           5779.64 795.56 122.70 111.78  88.66   
3                     True           1453.58 261.80  59.22  51.97  37.37   
4                    False            365.82  33.22  19.34  18.66   9.96   
5                     True            369.51  33.37  19.54  18.88   9.98   
6                    False           1512.48 241.27  40.98  38.99  26.36   
7                     True            368.94  72.67  21.54  20.39  11.22   

          Queue pushs [$\cdot 10^3$]                       
potential                       Zero     ALT (C)CH/Oracle  
0                            4493.97  292.01       125.16  
1                            4491.56  290.92       125.08  
2                           16042.82 1688.60       259.78  
3                            4491.56  624.25       116.71  
4                             916.15   57.27        23.60  
5                             908.55   56.09        23.25  
6                            3317.81  334.90        44.67  
7                             908.55  123.77        20.72

In [62]:
lines = pruning.to_latex(escape=False, index=False, multicolumn_format='c', column_format='c@{\hskip6pt}c@{\hskip3pt}crrrrrrrr').split("\n")

lines = lines[0:2] + [
    R' &  &  & \multicolumn{5}{c}{Running time [ms]} & \multicolumn{3}{c}{Queue pushs [$\cdot 10^3$]} \\ \cmidrule(lr){4-8} \cmidrule(lr){9-11}',
    R'Low Deg.      & Bidirectional & New & \multirow{2}{*}{Zero} & \multirow{2}{*}{ALT} & \multirow{2}{*}{CH} & \multirow{2}{*}{CCH} & \multirow{2}{*}{Oracle} & \multirow{2}{*}{Zero} & \multirow{2}{*}{ALT} & (C)CH/ \\',
    R'Opt. & Potential     & Pruning  & & & & & & & & Oracle \\'
] + lines[4:9] + ['\\addlinespace'] + lines[9:]

output = "\n".join(lines) + "\n"
output = output.replace('Bidrectional Dijkstra Query', R'\xmark')
output = output.replace('Virtual Topocore Bidirectional Core Query', R'\cmark')
output = output.replace('True', '\\cmark')
output = output.replace('False', '\\xmark')
output = re.sub(re.compile('([0-9]{3}(?=[0-9]))'), '\\g<0>,\\\\', output[::-1])[::-1]

print(output)

\begin{tabular}{c@{\hskip6pt}c@{\hskip3pt}crrrrrrrr}
\toprule
 &  &  & \multicolumn{5}{c}{Running time [ms]} & \multicolumn{3}{c}{Queue pushs [$\cdot 10^3$]} \\ \cmidrule(lr){4-8} \cmidrule(lr){9-11}
Low Deg.      & Bidirectional & New & \multirow{2}{*}{Zero} & \multirow{2}{*}{ALT} & \multirow{2}{*}{CH} & \multirow{2}{*}{CCH} & \multirow{2}{*}{Oracle} & \multirow{2}{*}{Zero} & \multirow{2}{*}{ALT} & (C)CH/ \\
Opt. & Potential     & Pruning  & & & & & & & & Oracle \\
\midrule
              \xmark &   Average &            \xmark &           1\,441.41 & 126.46 &  62.61 &  53.91 &  37.29 &                    4\,493.97 &  292.01 &       125.16 \\
              \xmark &   Average &             \cmark &           1\,451.96 & 128.20 &  62.48 &  54.28 &  38.89 &                    4\,491.56 &  290.92 &       125.08 \\
              \xmark & Symmetric &            \xmark &           5\,779.64 & 795.56 & 122.70 & 111.78 &  88.66 &                   16\,042.82 & 1\,688.60 &       259.78 \\
       

### Min key is Quatsch

In [7]:
bidir_switch_strat = queries.loc[lambda x: x['bidir_pot'] != 'unidir'] \
    .loc[lambda x: x['algo'].str.contains('Topocore')] \
    .loc[lambda x: ~(((x['bidir_pot'] == 'Average') & (x['improved_pruning'] == True)) | ((x['bidir_pot'] == 'Symmetric') & (x['improved_pruning'] == False)))] \
    .groupby(['potential', 'experiment', 'factor', 'probability', 'bidir_pot', 'choose_direction_strategy'])[['running_time_ms', 'num_queue_pushs_k']].mean().unstack(0) \
    .reindex(columns=['Zero', 'ALT', 'CH', 'CCH', 'Oracle'], level=1) \
    .reindex(index=['weight_scale', 'probabilistic_scale_by_speed'], level=0) \
    .reindex(index=[1.0], level=2)

bidir_switch_strat[('num_queue_pushs_k', '(C)CH/Oracle')] = bidir_switch_strat[('num_queue_pushs_k', 'CH')]
bidir_switch_strat = bidir_switch_strat.drop([('num_queue_pushs_k', 'CH'), ('num_queue_pushs_k', 'CCH'), ('num_queue_pushs_k', 'Oracle')], axis = 'columns')

bidir_switch_strat = bidir_switch_strat.rename(columns={
        'running_time_ms': 'Running time [ms]',
        'num_queue_pushs_k': 'Queue pushs [$\\cdot 10^3$]',
    }) \
    .rename(index={
        'alternating': 'Alternating',
        'min_key': 'Min. Key',
    })

bidir_switch_strat.loc[('probabilistic_scale_by_speed', 1.5, 1.0), '$w_q$'] = '< 80kph'
bidir_switch_strat.loc[('weight_scale', 1.0, 1.0), '$w_q$'] = 'unmodified'
bidir_switch_strat.loc[('weight_scale', 1.05, 1.0), '$w_q$'] = '*1.05'
bidir_switch_strat = bidir_switch_strat.reset_index(level=[0,1,2], drop=True).reset_index().set_index(['$w_q$', 'bidir_pot', 'choose_direction_strategy'])
bidir_switch_strat

Running time [ms]              \
potential                                                   Zero   ALT    CH   
$w_q$      bidir_pot choose_direction_strategy                                 
unmodified Average   Alternating                          373.18 12.83  0.79   
                     Min. Key                             406.35 13.68  1.44   
           Symmetric Alternating                          376.72 40.19  0.69   
                     Min. Key                             427.51 50.46  1.77   
*1.05      Average   Alternating                          365.82 33.22 19.34   
                     Min. Key                             391.70 38.06 21.76   
           Symmetric Alternating                          368.94 72.67 21.54   
                     Min. Key                             394.38 84.84 27.28   
< 80kph    Average   Alternating                          361.83 19.50 10.92   
                     Min. Key                             391.47 31.65 21.05   
           Symmetric Alternating                          364.55 37.33 11.89   
                     Min. Key                             395.04 54.90 23.36   

                                                             \
potential                                        CCH Oracle   
$w_q$      bidir_pot choose_direction_strategy                
unmodified Average   Alternating                1.13   0.18   
                     Min. Key                   1.75   0.56   
           Symmetric Alternating                0.92   0.19   
                     Min. Key                   1.99   0.83   
*1.05      Average   Alternating               18.66   9.96   
                     Min. Key                  20.44  11.30   
           Symmetric Alternating               20.39  11.22   
                     Min. Key                  24.64  14.53   
< 80kph    Average   Alternating               10.94   5.34   
                     Min. Key                  20.10  11.00   
           Symmetric Alternating               11.75   6.00   
                     Min. Key                  22.48  12.54   

                                               Queue pushs [$\cdot 10^3$]  \
potential                                                            Zero   
$w_q$      bidir_pot choose_direction_strategy                              
unmodified Average   Alternating                                   916.15   
                     Min. Key                                      986.40   
           Symmetric Alternating                                   908.55   
                     Min. Key                                      978.62   
*1.05      Average   Alternating                                   916.15   
                     Min. Key                                      986.41   
           Symmetric Alternating                                   908.55   
                     Min. Key                                      978.63   
< 80kph    Average   Alternating                                   845.06   
                     Min. Key                                      917.13   
           Symmetric Alternating                                   836.44   
                     Min. Key                                      908.12   

                                                                    
potential                                         ALT (C)CH/Oracle  
$w_q$      bidir_pot choose_direction_strategy                      
unmodified Average   Alternating                23.08         0.60  
                     Min. Key                   26.39         1.15  
           Symmetric Alternating                76.61         0.57  
                     Min. Key                   99.62         1.44  
*1.05      Average   Alternating                57.27        23.60  
                     Min. Key                   67.65        26.42  
           Symmetric Alternating               123.77        20.72  
                     Min. Key                  145.28 

In [51]:
table = bidir_switch_strat.reset_index()
table.loc[1:3, 'probability'] = ''
table.loc[5:9, 'probability'] = ''

lines = table.to_latex(escape=False, index=False, multicolumn_format='c', column_format='cccrrrrrrrr').split("\n")

lines = lines[0:2] + [
    R' &  &  & \multicolumn{5}{c}{Running time [ms]} & \multicolumn{3}{c}{Queue pushs [$\cdot 10^3$]} \\ \cmidrule(lr){4-8} \cmidrule(lr){9-11}'
    R'\multirow{2}{*}{$w_q$} & Bidirectional & Choose    & \multirow{2}{*}{Zero} & \multirow{2}{*}{ALT} & \multirow{2}{*}{CH} & \multirow{2}{*}{CCH} & \multirow{2}{*}{Oracle} & \multirow{2}{*}{Zero} & \multirow{2}{*}{ALT} & (C)CH/ \\',
    R' & Potential     & Direction & & & & & & & & Oracle \\'
] + lines[4:9] + ['\\addlinespace'] + lines[9:]

output = "\n".join(lines) + "\n"
output = output.replace('> 80kph', R'\multirow{4}{*}{\rotatebox[origin=c]{90}{\shortstack{$w_{\ell} \cdot 1.5$ if\\ $v >$ 80kph}}}')
output = output.replace('< 80kph', R'\multirow{4}{*}{\rotatebox[origin=c]{90}{\shortstack{$w_{\ell} \cdot 1.5$ if\\ $v <$ 80kph}}}')
output = re.sub(re.compile('([0-9]{3}(?=[0-9]))'), '\\g<0>,\\\\', output[::-1])[::-1]

print(output)

\begin{tabular}{cccrrrrrrrr}
\toprule
 &  &  & \multicolumn{5}{c}{Running time [ms]} & \multicolumn{3}{c}{Queue pushs [$\cdot 10^3$]} \\ \cmidrule(lr){4-8} \cmidrule(lr){9-11}\multirow{2}{*}{$w_q$} & Bidirectional & Choose    & \multirow{2}{*}{Zero} & \multirow{2}{*}{ALT} & \multirow{2}{*}{CH} & \multirow{2}{*}{CCH} & \multirow{2}{*}{Oracle} & \multirow{2}{*}{Zero} & \multirow{2}{*}{ALT} & (C)CH/ \\
 & Potential     & Direction & & & & & & & & Oracle \\
\midrule
    \multirow{4}{*}{\rotatebox[origin=c]{90}{\shortstack{$w_{\ell} \cdot 1.5$ if\\ $v >$ 80kph}}} &   Average &               Alternating &            373.50 & 270.89 & 349.91 & 328.52 & 202.29 &                     987.74 & 414.43 &       387.79 \\
            &   Average &                  Min. Key &            394.26 & 276.73 & 365.02 & 334.01 & 210.72 &                    1\,056.16 & 437.39 &       405.63 \\
            & Symmetric &               Alternating &            371.30 & 274.22 & 355.13 & 333.40 & 219.14 &        

### Bidir Effectiveness

In [67]:
bidir = queries.loc[lambda x: x['algo'].str.contains('Topocore')] \
    .loc[lambda x: x['choose_direction_strategy'] != 'min_key'] \
    .loc[lambda x: ~(((x['bidir_pot'] == 'Average') & (x['improved_pruning'] == True)) | ((x['bidir_pot'] == 'Symmetric') & (x['improved_pruning'] == False)))] \
    .groupby(['potential', 'experiment', 'factor', 'probability', 'bidir_pot'])[['running_time_ms', 'num_queue_pushs_k']].mean().unstack(0) \
    .reindex(columns=['Zero', 'ALT', 'CH', 'CCH', 'Oracle'], level=1) \
    .reindex(index=['weight_scale', 'probabilistic_scale_by_speed'], level=0) \
    .reindex(index=[1.0, 0.0], level=2) \
    .reindex(index=['unidir', 'Average', 'Symmetric'], level=3)

bidir[('num_queue_pushs_k', '(C)CH/Oracle')] = bidir[('num_queue_pushs_k', 'CH')]
bidir = bidir.drop([('num_queue_pushs_k', 'CH'), ('num_queue_pushs_k', 'CCH'), ('num_queue_pushs_k', 'Oracle')], axis = 'columns')

bidir = bidir.rename(columns={
    'running_time_ms': 'Running time [ms]',
    'num_queue_pushs_k': 'Queue pushs [$\\cdot 10^3$]',
}, index={'unidir': 'Unidirectional'})

bidir.loc[('probabilistic_scale_by_speed', 1.5, 0.0), '$w_q$'] = '> 80kph'
bidir.loc[('probabilistic_scale_by_speed', 1.5, 1.0), '$w_q$'] = '< 80kph'
bidir.loc[('weight_scale', 1.0, 1.0), '$w_q$'] = 'unmodified'
bidir.loc[('weight_scale', 1.05, 1.0), '$w_q$'] = '*1.05'
bidir = bidir.reset_index(level=[0,1,2], drop=True).reset_index().set_index(['$w_q$', 'bidir_pot'])
bidir

/usr/lib/python3.9/site-packages/IPython/core/interactiveshell.py:2901: PerformanceWarning: indexing past lexsort depth may impact performance.
  result = self._run_cell(
/usr/lib/python3.9/site-packages/IPython/core/async_helpers.py:68: PerformanceWarning: indexing past lexsort depth may impact performance.
  coro.send(None)


Running time [ms]                              \
potential                              Zero    ALT     CH    CCH Oracle   
$w_q$      bidir_pot                                                      
unmodified Unidirectional            584.87  43.02   0.47   0.64   0.16   
           Average                   373.18  12.83   0.79   1.13   0.18   
           Symmetric                 376.72  40.19   0.69   0.92   0.19   
*1.05      Unidirectional            588.55  90.90  17.19  16.25  11.45   
           Average                   365.82  33.22  19.34  18.66   9.96   
           Symmetric                 368.94  72.67  21.54  20.39  11.22   
< 80kph    Unidirectional            637.24  96.62  21.78  21.37  14.62   
           Average                   361.83  19.50  10.92  10.94   5.34   
           Symmetric                 364.55  37.33  11.89  11.75   6.00   
> 80kph    Unidirectional            546.91 327.19 305.81 294.09 221.13   
           Average                   373.50 270.89 349.91 328.52 202.29   
           Symmetric                 371.30 274.22 355.13 333.40 219.14   

                          Queue pushs [$\cdot 10^3$]                      
potential                                       Zero    ALT (C)CH/Oracle  
$w_q$      bidir_pot                                                      
unmodified Unidirectional                    1674.35  96.21         0.66  
           Average                            916.15  23.08         0.60  
           Symmetric                          908.55  76.61         0.57  
*1.05      Unidirectional                    1674.36 179.63        26.78  
           Average                            916.15  57.27        23.60  
           Symmetric                          908.55 123.77        20.72  
< 80kph    Unidirectional                    1674.26 171.02        36.54  
           Average                            845.06  34.03        13.25  
           Symmetric                          836.44  57.93        11.53  
> 80kph    Unidirectional                    1677.02 599.76       456.93  
           Average                            987.74 414.43       387.79  
           Symmetric                          981.38 424.35       377.42

In [69]:
lines = bidir.to_latex(escape=False, multicolumn_format='c', column_format='ccrrrrrrrr').split("\n")

lines = lines[0:2] + [
    R' &  & \multicolumn{5}{c}{Running time [ms]} & \multicolumn{3}{c}{Queue pushs [$\cdot 10^3$]} \\ \cmidrule(lr){3-7} \cmidrule(lr){8-10}'
    R'\multirow{2}{*}{$w_q$} & & \multirow{2}{*}{Zero} & \multirow{2}{*}{ALT} & \multirow{2}{*}{CH} & \multirow{2}{*}{CCH} & \multirow{2}{*}{Oracle} & \multirow{2}{*}{Zero} & \multirow{2}{*}{ALT} & (C)CH/ \\',
    R' & & & & & & & & & Oracle \\'
] + lines[5:9] + ['\\addlinespace'] + lines[9:12] + ['\\addlinespace'] + lines[12:15] + ['\\addlinespace'] + lines[15:]

output = "\n".join(lines) + "\n"
output = output.replace('> 80kph', R'\multirow{3}{*}{\shortstack{$w_{\ell} \cdot 1.5$ if\\ $v >$ 80kph}}')
output = output.replace('< 80kph', R'\multirow{3}{*}{\shortstack{$w_{\ell} \cdot 1.5$ if\\ $v <$ 80kph}}')
output = output.replace('unmodified', R'\multirow{3}{*}{$w_{\ell}$}')
output = output.replace('*1.05', R'\multirow{3}{*}{$w_{\ell} \cdot 1.05$}')
output = re.sub(re.compile('([0-9]{3}(?=[0-9]))'), '\\g<0>,\\\\', output[::-1])[::-1]

print(output)

\begin{tabular}{ccrrrrrrrr}
\toprule
 &  & \multicolumn{5}{c}{Running time [ms]} & \multicolumn{3}{c}{Queue pushs [$\cdot 10^3$]} \\ \cmidrule(lr){3-7} \cmidrule(lr){8-10}\multirow{2}{*}{$w_q$} & & \multirow{2}{*}{Zero} & \multirow{2}{*}{ALT} & \multirow{2}{*}{CH} & \multirow{2}{*}{CCH} & \multirow{2}{*}{Oracle} & \multirow{2}{*}{Zero} & \multirow{2}{*}{ALT} & (C)CH/ \\
 & & & & & & & & & Oracle \\
\midrule
\multirow{3}{*}{$w_{\ell}$} & Unidirectional &            584.87 &  43.02 &   0.47 &   0.64 &   0.16 &                    1\,674.35 &  96.21 &         0.66 \\
        & Average &            373.18 &  12.83 &   0.79 &   1.13 &   0.18 &                     916.15 &  23.08 &         0.60 \\
        & Symmetric &            376.72 &  40.19 &   0.69 &   0.92 &   0.19 &                     908.55 &  76.61 &         0.57 \\
\addlinespace
\multirow{3}{*}{$w_{\ell} \cdot 1.05$} & Unidirectional &            588.55 &  90.90 &  17.19 &  16.25 &  11.45 &                    1\,674.36 & 179.63 & 